# Recommendation using corelations

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [3]:
def get_data():
    column_names = ['user_id', 'item_id', 'rating', 'timestamp']
    df = pd.read_csv('data/u.data', sep='\t', names=column_names)
    movie_titles = pd.read_csv("data/Movie_Id_Titles")
    movie_data = pd.merge(df,movie_titles,on='item_id')

    return movie_data


In [23]:
# Below function is for analysis purpose where you can decide how we can corelated data with other attributes
def plot(movie_data):
    
    ratings = pd.DataFrame(movie_data.groupby('title')['rating'].mean())
    ratings['num of ratings'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())
    # ratings = ratings[ratings['num of ratings'] >100]
    
#     plt.figure(figsize=(10,4))
#     ratings['num of ratings'].hist(bins=70)
    
#     plt.figure(figsize=(10,4))
#     ratings['rating'].hist(bins=70)
    
    moviemat = movie_data.pivot_table(index='user_id',columns='title',values='rating')
    return moviemat
moviemat = plot(movie_data)
moviemat.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Now let's create a matrix that has the user ids on one access and the movie title on another axis. 
# Each cell will then consist of the rating the user gave to that movie.
# Note there will be a lot of NaN values, because most people have not seen most of the movies.
def get_recommendation(movie_data, movie_name):
    # Rating average and num of rating is important because we have to corelated data to recommend the movie
    # as per rating and how many user given rating 
    ratings = pd.DataFrame(movie_data.groupby('title')['rating'].mean())
    ratings['num of ratings'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())
    
    # create pivot table for user/movie name and rating.
    moviemat = movie_data.pivot_table(index='user_id',columns='title',values='rating')
    
    # Get movie data from pivot table 
    movie_user_ratings = moviemat[movie_name]
    
    # Corelate movie data with pivot table
    similar_to_movie = moviemat.corrwith(movie_user_ratings)
    corr_movie = pd.DataFrame(similar_to_movie,columns=['Correlation'])
    corr_movie.dropna(inplace=True)
    
    corr_movie = corr_movie.join(ratings['num of ratings'])
    data = corr_movie[corr_movie['num of ratings']>100].sort_values('Correlation',ascending=False)
    return data.head()


In [17]:
movie_data = get_data()
data = get_recommendation(movie_data, "Star Wars (1977)")


/home/rups/.virtualenvs/jupyiter_env/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/rups/.virtualenvs/jupyiter_env/lib/python3.6/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation,num of ratings
title,,
Star Wars (1977),1.000000,584
"Empire Strikes Back, The (1980)",0.748353,368
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Austin Powers: International Man of Mystery (1997),0.377433,130
